<a href="https://colab.research.google.com/github/sebavassou/chatbot_fila_into/blob/main/caderno/chatbot_INTO02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain-community
!pip install streamlit
!pip install pypdf
!pip install dill


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [5]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 13.4 MB/s eta 0:00:00


In [6]:
from langchain.document_loaders import PyPDFLoader

# Carregar o PDF using the raw URL
loader = PyPDFLoader("https://raw.githubusercontent.com/sebavassou/chatbot_fila_into/main/dados/Cartilha-Funcionamento-da-Lista-de-Espera-do-Into_web2.pdf")
documents = loader.load()


In [7]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [8]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Criar embeddings com SentenceTransformer
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Indexar os documentos com FAISS
vector_store = FAISS.from_documents(docs, embeddings)


In [18]:
from google.colab import userdata
# Before attempting to get the secret, you need to create it:
# userdata.set('secretName', 'your_actual_secret_value') # Replace with your actual secret
# OR comment out this line as it might be redundant
# userdata.get('secretName')

from dotenv import load_dotenv  # Import the load_dotenv function
load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")  # Try using .get() to avoid KeyError
n_documentos = 3

In [19]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)



ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-4o-mi...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
query = "EM que casos a cirurgia pode ser suspensa?"
result = qa_chain({"query": query})

print("Resposta:", result["result"])
print("\nFontes:")
for doc in result["source_documents"]:
    print(doc.page_content[:10])  # Exibe os primeiros 500 caracteres da fonte